### Imports

In [1]:
from matplotlib import rc
from matplotlib import colors
from matplotlib.patches import FancyArrowPatch
rc('animation', html='jshtml')

from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
import abc
from numpy.typing import NDArray

from scipy import stats

### Game Plotter.
Plots are generated with game history and actions of agents. You should not need to edit this code.

In [2]:
#generate a fake game with walls and agents
walls = np.random.binomial(1, 0.2, (8,8))
test_map = [np.random.randint(0,4, (8,8)) for i in range(10)]
for i in test_map:
  i[i==1] = 0
  i[walls== 1] = 1
# [empty, wall, friendly, enemy]

#generate actions taken by agents
action_hist = [(np.random.randint(0, 25, (8,8)),np.random.randint(11, 13, (8,8))) for _ in range(10)]

# RGB color definitions
color_empty = (0.98, 0.98, 0.98)  # Slightly off-white for empty cells
color_wall = (0.3, 0.3, 0.3)  # Dark grey for walls
color_p1 = (0.8, 0.2, 0.2)  # Less saturated red for player 1
color_p2 = (0.2, 0.2, 0.8)  # Less saturated blue for player 2
color_p1_attack = (1.0, 0.5, 0.5)  # Light red for player 1 attack
color_p2_attack = (0.5, 0.5, 1.0)  # Light blue for player 2 attack


import matplotlib.pyplot as plt
from matplotlib.patches import Arc, RegularPolygon
import numpy as np
from numpy import radians as rad

#draw the circle
def drawCirc(ax,radius,centX,centY,angle_,theta2_, line_width = 3,color_='black'):
    #========Line
    arc = Arc([centX,centY],radius,radius,angle=angle_,
          theta1=0,theta2=theta2_,capstyle='round',linestyle='-',lw=line_width,color=color_)
    ax.add_patch(arc)


# draw the attack lines from agents
def calculate_attack_lines(frame_matrix, action_matrix, player_number):
    lines = []
    for i in range(frame_matrix.shape[0]):
        for j in range(frame_matrix.shape[1]):
            if frame_matrix[i, j] == player_number:
                action = int(action_matrix[i, j])
                direction = np.unravel_index(action, (5, 5))
                direction = (direction[0] - 2, direction[1] - 2)  # Adjust to get -2 to 2 range
                start = (j, i)  # Center of the cell

                if direction == (0, 0):  # Self-attack
                    lines.append((start, 'self'))
                else:
                    end = (start[0] + direction[1], start[1] + direction[0])  # Calculate end point
                    lines.append((start, end))

    return lines

#frame function - used to draw all elements in each frame
def frame(w, im, p1_lines, p2_lines, ax, cmap, norm):
    height, width = w.shape
    im.set_data(w)
    ax.clear()
    ax.imshow(w, cmap=cmap, norm=norm, origin = 'upper')


    # Draw attack lines for both players
    for line in p1_lines:
        if line[1] == 'self':  # Self-attack
            drawCirc(ax,0.35,line[0][0],line[0][1],90,270,line_width = 3,color_=color_p1_attack)
        else:
            ax.annotate('', xy=line[1], xytext=line[0],
                        arrowprops=dict(arrowstyle="->, head_length=.9, head_width=0.3", color=color_p1_attack, linewidth=3), annotation_clip = False)
    # Repeat for player 2
    for line in p2_lines:
        if line[1] == 'self':  # Self-attack
            drawCirc(ax,0.35,line[0][0],line[0][1],90,270,line_width = 3,color_=color_p2_attack)
        else:
            ax.annotate('', xy=line[1], xytext=line[0],
                        arrowprops=dict(arrowstyle="->, head_length=.9, head_width=0.3", color=color_p2_attack, linewidth=3), annotation_clip = False)

    # Rest of the function...

    # Add grid lines for visual separation of cells
    ax.set_xticks(np.arange(-0.5, width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, height, 1), minor=True)
    ax.grid(which="minor", color="grey", linestyle='-', linewidth=0.5)
    ax.tick_params(which="minor", size=0)

    return im

def anim_builder(frames, action_hist, fig_size = (4,4)):
    """
    Builds the animation with attack lines if provided.
    """
    fig, ax = plt.subplots(figsize=(6,6))
    cmap = colors.ListedColormap([color_empty, color_wall, color_p1, color_p2])
    bounds = [-0.5, 0.5, 1.5, 2.5, 3.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    im = ax.imshow(frames[0], cmap=cmap, norm=norm)
    #h,w = frames[0].shape

    def frame_with_lines(frame_num):
        if frame_num+1 < len(action_hist):
          p1_lines = calculate_attack_lines(frames[frame_num], action_hist[frame_num+1][0], 2) if action_hist else []
          p2_lines = calculate_attack_lines(frames[frame_num], action_hist[frame_num+1][1], 3) if action_hist else []
        else:
          p1_lines = []
          p2_lines = []
        return frame(frames[frame_num], im, p1_lines, p2_lines, ax, cmap, norm)

    anim = animation.FuncAnimation(fig, frame_with_lines, frames=len(frames), blit=False, repeat=True)
    plt.close(fig)
    return anim

# Create the animation with attack lines
anim = anim_builder(test_map, action_hist)
#display the result in the notebook
anim

In [8]:
anim.save('test_anim.gif', writer='pillow', fps=5)

In [7]:
#call for getting adjacent cell features
# given x,y, return cells in range
def get_cell_feats(state_mat, y, x, n=1):
    rows, cols = state_mat.shape

    if not 0 <= x < cols:
        raise ValueError('Invalid X coordinate')
    if not 0 <= y < rows:
        raise ValueError('Invalid Y coordinate')

    y_indices = [(y+i) % rows for i in range(-n, n+1)]
    x_indices = [(x+i) % cols for i in range(-n, n+1)]

    return state_mat[np.ix_(y_indices, x_indices)]

## Agent Class + Example Agents. Feel Free to modify or train off of!

### The Agent Superclass.
Your code will need to extend this class.

In [3]:
class Agent():
  __metaclass__ = abc.ABCMeta

  def __init__(self,
                 obs_onehot: bool = False,
                 attack_map: bool = False,
                 attack_map_logits: bool = False,
                 ):
        """
        :param obs_onehot: Determines whether the observation that is received is one-hot encoded or not
        :type obs_onehot: bool
        :param attack_map: Determines whether you pass in a probability map for each cell's attack
        (4 dimensional array for your total action)
        :type attack_map: bool
        :param attack_map_logits: If you pass in a map for each cell's attack, this determines whether it
        is a logit or probability map
        :type attack_map_logits: bool
        """
        self.obs_onehot = obs_onehot
        self.attack_map = attack_map
        self.attack_map_logits = attack_map_logits

  @abc.abstractmethod
  def policy(self, obs: NDArray) -> NDArray:
      raise NotImplementedError("Please implement the policy method!")

  # stores game data (left to you for training)
  def cache(self, args):
    pass


### NEAT Agent

Uses NEAT to find the best possible strategy

In [17]:
class neat_agent(Agent):
  desired_length = 2500

  def cache(self, step):
    state = step[0]
    next_state = step[1]
    action = step[2]
    reward = step[3]


  def __init__(self,
                 obs_onehot: bool = False,
                 attack_map: bool = False,
                 attack_map_logits: bool = False,
                 ):
        self.obs_onehot = obs_onehot
        self.attack_map = attack_map
        self.attack_map_logits = attack_map_logits
        self.adj_mat_ind = None
        return

  def policy(self, state_mat, net, action_space= None, obs_space = None):
    attack_mat = np.zeros(state_mat.shape).astype(int)

    # Set the desired length (2500 in this case)
    desired_length = 2500

    # Flatten the state_mat
    flattened_state = state_mat.flatten()

    # Calculate the number of zeros to add
    zeros_to_add = max(0, desired_length - flattened_state.size)

    # Pad the flattened array with zeros
    padded_state = np.pad(flattened_state, (0, zeros_to_add), mode='constant')

    padded = np.append(padded_state, [state_mat.shape[0], state_mat.shape[1], 0, 0])

    rows, cols = state_mat.shape
    for i in range(rows):
      for j in range(cols):
        if (state_mat[i, j] == 2):
          padded[2500] = i;
          padded[2501] = j;
          output = net.activate((padded))
          bit0 = 1 if output[0] > 0.5 else 0
          bit1 = 1 if output[1] > 0.5 else 0
          bit2 = 1 if output[2] > 0.5 else 0
          bit3 = 1 if output[3] > 0.5 else 0
          bit4 = 1 if output[4] > 0.5 else 0
          ret = bit0*16 + bit1*8 + bit2*4 + bit3*2 + bit4
          if (ret > 24):
            ret = 12
          attack_mat[i, j] = ret

    return attack_mat.astype(int)

### Random Agent

Chooses a random point to attack

In [18]:
class rand_agent(Agent):

  def policy(self, state_mat, net, action_space = None, obs_space = None):
    h = state_mat.shape[0]
    w = state_mat.shape[1]
    attack_mat =np.random.uniform(0,25,(h,w)).astype(int)

    return attack_mat.astype(int)

### First Enemy Agent

Attacks the first enemy it sees (So starting with top left going down)

In [19]:
class first_enemy_agent(Agent):
  def simple_adj(self, feat_mat):
    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >= 3)[:,0]
    empty_list = np.argwhere(feat_mat == 0)[:,0]

    if len(enemy_list)>=1: #if there is at least one enemy
      return enemy_list[0]
    elif len(empty_list)>=1: #if there is at least one empty
      return empty_list[0]
    else:
      return 12

  #get the adjacent features for a given x,y cell
  def get_cell_feats(self,state_mat, x,y, n = 2):
    h,w = state_mat.shape
    adj_width = 2*n+1

    xind,yind = np.indices((adj_width,adj_width))
    xind = (xind -n +x)%h
    yind = (yind -n +y)%w

    #gets a matrix of the adjacent cells by 1
    adj_vals = state_mat[xind,yind]
    return adj_vals

  def policy(self, state_mat, net, action_space = None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape)

    for i in range(attack_mat.shape[0]):
      for j in range(attack_mat.shape[1]):
        attack_mat[i,j] = self.simple_adj(self.get_cell_feats(state_mat, i,j,2)) #get cell feats returns adjacent cells

    return attack_mat.astype(int)

### Random Enemy Agent

Chooses a random enemy within reach and attacks it

In [20]:
class rand_enemy_agent(Agent):

  #function for individual cell logic
  def rand_adj(self, feat_mat):

    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >=3)[:,0] #enemies are 3 or higher
    empty_list = np.argwhere(feat_mat == 0)[:,0] #empties are always 0

    if len(enemy_list)>=1:
      return np.random.choice(enemy_list,1)[0]
    if len(empty_list)>=1:
      return np.random.choice(empty_list,1)[0]
    else:
      return 12

  #Gets the values of the cells adjacent to a given x, y
  def get_cell_feats(self,state_mat, x,y, n = 2):
    h,w = state_mat.shape
    adj_width = 2*n+1

    xind,yind = np.indices((adj_width,adj_width))
    xind = (xind -n +x)%h
    yind = (yind -n +y)%w

    #gets a matrix of the adjacent cells by 1
    adj_vals = state_mat[xind,yind]
    return adj_vals

  #the global policy for this agent
  def policy(self, state_mat, net, action_space = None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape)
    for i in range(attack_mat.shape[0]):
      for j in range(attack_mat.shape[1]):
        attack_mat[i,j] = self.rand_adj(self.get_cell_feats(state_mat, i,j, 2)) #get cell feats returns adjacent cells

    return attack_mat.astype(int)

### Defensive Random Enemy Agent


In [ ]:
class def_rand_agent(Agent):

  #function for individual cell logic
  def rand_adj(self, feat_mat):

    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >=3)[:,0] #enemies are 3 or higher
    empty_list = np.argwhere(feat_mat == 0)[:,0] #empties are always 0
    friendly_list = np.argwhere(feat_mat == 2)[:,0] #friendlies are always 0

    if len(enemy_list)>=1:
      return np.random.choice(enemy_list,1)[0]
    if len(empty_list)>=1:
      if len(friendly_list) >=12:
        return np.random.choice(friendly_list,1)[0]
      return np.random.choice(empty_list,1)[0]
    else:
      return 12

  #Gets the values of the cells adjacent to a given x, y
  def get_cell_feats(self,state_mat, x,y, n = 2):
    h,w = state_mat.shape
    adj_width = 2*n+1

    xind,yind = np.indices((adj_width,adj_width))
    xind = (xind -n +x)%h
    yind = (yind -n +y)%w

    #gets a matrix of the adjacent cells by 1
    adj_vals = state_mat[xind,yind]
    return adj_vals

  #the global policy for this agent
  def policy(self, state_mat, action_space = None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape)
    for i in range(attack_mat.shape[0]):
      for j in range(attack_mat.shape[1]):
        attack_mat[i,j] = self.rand_adj(self.get_cell_feats(state_mat, i,j, 2)) #get cell feats returns adjacent cells

    return attack_mat.astype(int)

### Priority Agent

This agent prioritizes low population areas when expanding.

In [ ]:
from scipy import signal

class priority_agent(Agent):

  def __init__(self,
                 obs_onehot: bool = False,
                 attack_map: bool = False,
                 attack_map_logits: bool = False,
                 ):
        self.obs_onehot = obs_onehot
        self.attack_map = attack_map
        self.attack_map_logits = attack_map_logits
        self.adj_mat_ind = None
        return

  #gets the adjacent x,y coordinates to every friendly agent on the board
  def get_adj_indices(self, h, w , n = 1):
    adj_width = 2*n+1
    X,Y = np.indices((h,w))
    x,y = np.indices((adj_width, adj_width))
    x-=n
    y-=n
    X = X.reshape(h,w,1,1) + x
    Y = Y.reshape(h,w,1,1) + y

    X = X%h
    Y = Y%w

    X = X.reshape(h,w,-1)
    Y = Y.reshape(h,w,-1)
    return X, Y

  #gets the population mat from the state_mat
  def get_pop(self, state_mat):
    abs_map = (state_mat>=2) #get locations where there is an agent
    sum_mat = signal.convolve2d(abs_map, np.ones((3,3)), boundary = 'wrap', mode = 'same')
    return sum_mat

  #maximum population among all adjacent spaces to all friendly agents
  def get_max_pop(self, pop_mat):
    adj_pop = pop_mat[self.X_ind, self.Y_ind]
    max_pop = np.max(adj_pop, axis = 2)

    return max_pop


  def smart_adj(self, feat_mat, pop_mat): #helper function from individual cell perspective

    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >=3)[:,0]
    empty_list = np.argwhere((feat_mat == 0) * (pop_mat<=6))[:,0]

    if len(enemy_list)>=1: #if theres an enemy choose a random enemy
      if np.random.uniform()<0.8:
        return np.random.choice(enemy_list,1)[0]
      return 12

    elif len(empty_list)>=1: # if theres an empty list, choose the option with the lowest population
      enemy_ind = np.argmin(pop_mat[empty_list])
      return empty_list[enemy_ind]
    else:
      return 12

  def policy(self, state_mat, action_space= None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape).astype(int)

    #get the indices for adjacent cells
    if self.adj_mat_ind is None:
      h,w = state_mat.shape
      self.X_ind, self.Y_ind = self.get_adj_indices(h,w,n = 2)


    X_ind = self.X_ind
    Y_ind = self.Y_ind

    pop_mat = self.get_pop(state_mat) #get the one distance population map
    max_pop = self.get_max_pop(pop_mat)

    for i in range(attack_mat.shape[0]): #for each row
      for j in range(attack_mat.shape[1]): #for each column

        current = state_mat[i,j]
        if current == 2:#only run if theres an ally cell here.

          adj_x = X_ind[i,j,:]
          adj_y = Y_ind[i,j,:]

          adj_pop = max_pop[adj_x, adj_y]
          feat_mat = state_mat[adj_x, adj_y]

          ind = self.smart_adj(feat_mat, adj_pop) #get cell feats returns adjacent cells
          attack_mat[i,j] = ind

          a_x = adj_x[ind] #x pos being attacked
          a_y = adj_y[ind] #y pos being attacked

          target = state_mat[a_x, a_y]

          if target == 0: #go back and edit the population map.
            state_mat[a_x, a_y] = 2

            adj_x = X_ind[a_x,a_y,:]
            adj_y = Y_ind[a_x,a_y,:]

            pop_mat[adj_x, adj_y]+=1
            max_pop = self.get_max_pop(pop_mat)

    return attack_mat.astype(int)

## Environment Code.

In [35]:
from scipy import signal


class cgol:
  def  __init__(self, agent_1, agent_2, board_h = 10, board_w = 10, attack_width = 5, max_rounds = 100,map = None):

    self.board_h = board_h
    self.board_w = board_w
    self.view_width = attack_width
    self.view_height = attack_width
    self.max_rounds = max_rounds


    if map is None:
      self.map = np.zeros((self.board_h,self.board_w))
    else:
      self.map = map

    self.agent_1 = agent_1
    self.agent_2 = agent_2

    self.over_pop = 6

    #mapping for action int to
    x_view, y_view = np.indices((5,5))
    x_view-=2
    y_view-=2
    self.x_view = x_view.flatten()
    self.y_view = y_view.flatten()

    self.reset()

  def reset(self):
    self.state_mat = np.zeros((self.board_h,self.board_w)) + self.map

    start_num = 1
    #set starting locations for agent
    game_y = np.random.randint(0,self.board_h, start_num) #starting locations for agent_1_x
    game_x = np.random.randint(0,self.board_w, start_num) #starting locations for agent_1_y
    self.state_mat[game_y, game_x] = 2

    game_y = np.random.randint(0,self.board_h, start_num) #starting locations for agent_1_x
    game_x = np.random.randint(0,self.board_w, start_num) #starting locations for agent_1_y
    self.state_mat[game_y, game_x] = 3

    #recording history
    self.state_history = []
    self.action_history = []
    self.reward_history = []
    self.count = 0

  def get_state(self):
    return self.state_mat

  #moves from cords to attack to attack_map
  #action_map - map of board with actions assigned integer based on coord
  #ex:
  # [1 3 4 5 0 9]      [sum of adjent attacks]
  # [3 4 9 8 6 2]      [                     ]
  # [1 0 6 7 5 3] ---> [                     ]
  # [0 0 0 4 5 6]      [                     ]
  def convert_action_map(self, action_matrix, mask_matrix):

    rows, cols = action_matrix.shape

    # Convert action_matrix to direction coordinates
    directions = np.array(np.unravel_index(action_matrix.ravel(), (5, 5))).T - 2
    y_coords, x_coords = np.indices(action_matrix.shape)

    # Filter directions by mask
    masked_directions = directions[mask_matrix.ravel()]
    y_attacked = (y_coords.ravel()[mask_matrix.ravel()] + masked_directions[:, 0]) % rows
    x_attacked = (x_coords.ravel()[mask_matrix.ravel()] + masked_directions[:, 1]) % cols

    # Aggregate attacks on each cell
    result = np.zeros_like(action_matrix)
    np.add.at(result, (y_attacked, x_attacked), 1)

    return result

  #action_mat_1 - the action being played for this step by agent_1
  #action_mat_2 - the action being played for this step by agent_2
  #returns - reward, done

  # reward based off relative size
  def get_reward(self):
    p1_reward = np.count_nonzero(self.state_mat == 2)
    p2_reward = np.count_nonzero(self.state_mat == 3)
    if (p1_reward > p2_reward):
      p1_reward -= p2_reward
      p2_reward = p1_reward * -1
    else:
      p2_reward -= p1_reward
      p1_reward = p2_reward * -1

    return (p1_reward, p2_reward)

  #allys are always 2, so modify it here.
  def modify_view(self, state_mat, agent_num = 2):
    view = np.copy(state_mat)

    view[state_mat==agent_num] = 2
    view[state_mat==2] = agent_num
    return view

  def get_agent_views(self, state_mat=None):
    state_mat = state_mat if state_mat is not None else self.state_mat
    p1_view = self.modify_view(state_mat, 2)
    p2_view = self.modify_view(state_mat, 3)
    return p1_view, p2_view

  def calculate_winner(self):
    # Flattening the 2D array and then filtering out 0 and 1
    filtered_array = self.state_mat.flatten()
    filtered_array = filtered_array[(filtered_array != 0) & (filtered_array != 1)]

    # Finding the most common value in the filtered array
    mode = stats.mode(filtered_array)
    return mode

  def step(self, actions=(None, None), is_training=True):

    #print(actions)

    p1_action, p2_action = actions
    p1_view, p2_view = self.get_agent_views()
    ##retrieve adjent coordinates
    p1_mask = p1_view==2 #where p1 sees allys
    p2_mask = p2_view==2 #where p2 sees allys

    #translate adjent actions into sum attack matrices
    p1_sum = self.convert_action_map(p1_action, p1_mask)
    p2_sum = self.convert_action_map(p2_action, p2_mask)

    #add an always occuring self attack
    p1_sum += 1*(self.state_mat==2)
    p2_sum += 1*(self.state_mat==3)


    # Calculate total attacks for each cell.
    total_attacks = p1_sum + p2_sum

    rewardReturn = (0,0)

    with np.errstate(divide='ignore', invalid='ignore'):
        p1_win_mask = (total_attacks > 0)&(p1_sum == total_attacks)
        p2_win_mask = (total_attacks > 0)&(p2_sum == total_attacks)

        #print(p1_win_mask)

        both_nonzero_mask = (p1_sum > 0) & (p2_sum > 0)

        random_numbers = np.random.rand(*p1_sum.shape)
        p1_prob = p1_sum / (p1_sum + p2_sum)
        p1_prob[np.isnan(p1_prob)] = 0

        p1_prob_win = random_numbers < p1_prob
        p2_prob_win = random_numbers >= p1_prob
        p1_prob_win = p1_prob_win*both_nonzero_mask
        p2_prob_win = p2_prob_win*both_nonzero_mask

        original_state_mat = np.copy(self.state_mat)
        self.state_mat[p1_win_mask] = 2
        self.state_mat[p1_prob_win] = 2
        a1kills = np.count_nonzero(original_state_mat != self.state_mat)

        original_state_mat = np.copy(self.state_mat)
        self.state_mat[p2_win_mask] = 3
        self.state_mat[p2_prob_win] = 3
        a2kills = np.count_nonzero(original_state_mat != self.state_mat)
        if (a1kills > a2kills):
          a1kills -= a2kills
          a2kills = a1kills * -1
        else:
          a2kills -= a1kills
          a1kills = a2kills * -1
        a1kills *= 100
        a2kills *= 100

        # Convert the tuple to a list to make modifications
        modified_list = list(rewardReturn)

        # Change values in the list
        modified_list[0] = a1kills
        modified_list[1] = a2kills

        # Convert the list back to a tuple
        rewardReturn = tuple(modified_list)

    self.state_mat[self.map==1]=1

    ###ADD PUNISHMENT FOR OVERPOPULATION #################################
    original_state_mat = np.copy(self.state_mat)

    abs_map = (self.state_mat>=2) #get locations where there is an agent
    sum_mat = signal.convolve2d(abs_map, np.ones((3,3)), boundary = 'wrap', mode = 'same')
    excess_mat = sum_mat>self.over_pop
    self.state_mat = self.state_mat * (1-(1*excess_mat))

    self.state_mat[self.map==1]=1 #walls always equal 1

    # Count the changes where the original value was 2 or 3
    a1deaths = np.count_nonzero((original_state_mat == 2) & (self.state_mat != original_state_mat))
    a2deaths = np.count_nonzero((original_state_mat == 3) & (self.state_mat != original_state_mat))
    a1deaths *= -100
    a2deaths *= -100

    #####reward! modify as needed (your objective is to win)##############
    reward = self.get_reward()
    reward_list = list(reward)

    # Change values in the list
    reward_list[0] += rewardReturn[0]
    reward_list[1] += rewardReturn[1]

    # Convert the list back to a tuple
    reward = tuple(reward_list)

    #check if game end state is complete
    done = False
      #reward = 10000
    if np.max(self.state_mat) == 1:
      done = True
      #reward = -100
    if self.count >= self.max_rounds:
      done = True

    if not np.any(self.state_mat == 2):
      done = True
    if not np.any(self.state_mat == 3):
      done = True

    self.count+=1

    p1_view, p2_view = self.get_agent_views()
    return (p1_view, p2_view), reward, done

  #returns a history of the game
  def run_game(self, episodes = 10, training=True):
    episode_winner_record = []
    episodes_game_history = []
    episodes_action_history = []

    # set agent flags for training
    self.agent_1.is_training = training
    self.agent_2.is_training = training

    for episode in range(episodes):
      self.reset()
      while True:

          p1_view, p2_view = self.get_agent_views()

          # get attack actions from agent
          p1_action = self.agent_1.policy(p1_view)
          p2_action = self.agent_2.policy(p2_view)

          # Recieve state and reward from environment from chosen actions
          player_views_new, reward, done = self.step((p1_action, p2_action))
          p1_view_new, p2_view_new = player_views_new
          p1_reward, p2_reward = reward

          #print(self.action_history)
          # if training, record data
          if training:
            # example caching of data. Up to you to edit, modify, use, ect
            self.agent_1.cache((p1_view, p1_view_new, p1_action, p1_reward))
            self.agent_2.cache((p2_view, p2_view_new, p2_action, p2_reward))

          # End this episode when `done` is True
          if done:
            episode_winner_record.append(self.calculate_winner())
            episodes_game_history.append(self.state_history)
            episodes_action_history.append(self.action_history)
            break
    return episode_winner_record, episodes_game_history, episodes_action_history

    #evaluate genome
  def eval_genome(self, net1, net2, training=True):
    # set agent flags for training
    self.agent_1.is_training = training
    self.agent_2.is_training = training

    fitness_1 = 0;
    fitness_2 = 0;

    self.reset()
    while True:
        p1_view, p2_view = self.get_agent_views()

        # get attack actions from agent
        p1_action = self.agent_1.policy(p1_view, net1)
        p2_action = self.agent_2.policy(p2_view, net2)

        # Recieve state and reward from environment from chosen actions
        player_views_new, reward, done = self.step((p1_action, p2_action))
        # p1_view_new, p2_view_new = player_views_new
        p1_reward, p2_reward = reward

        # reward based off relative kills divided by 10 to reduce impact
        fitness_1 += (p1_reward/10)
        fitness_2 += (p2_reward/10)

        #print(self.action_history)
        # if training, record data
        # if training:
        #   # example caching of data. Up to you to edit, modify, use, ect
        #   self.agent_1.cache((p1_view, p1_view_new, p1_action, p1_reward))
        #   self.agent_2.cache((p2_view, p2_view_new, p2_action, p2_reward))

        # End this episode when `done` is True
        if done:
          winner = self.calculate_winner()
          if (winner == 2):
            fitness_1 += 10000
            fitness_2 -= 10000
          if (winner == 3):
            fitness_1 -= 10000
            fitness_2 += 10000
          break

    return fitness_1, fitness_2

## Test of the game

In [6]:
pip install neat-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
import neat
import os
import random
import pickle

In [22]:
def run(config_path):
  config = neat.config.Config(neat.DefaultGenome,
                              neat.DefaultReproduction,
                              neat.DefaultSpeciesSet,
                              neat.DefaultStagnation,
                              config_path)
  p = neat.Population(config) # set a population

  p.add_reporter(neat.StdOutReporter(True))
  stats = neat.StatisticsReporter()
  p.add_reporter(stats)

  winner = p.run(run_games,150) # set a fitness function to run X generations

  print('\nBest genome:\n{!s}'.format(winner))

In [37]:
GEN = 0
def run_games(genomes, config):
    global GEN
    GEN += 1

    agents = []
    ge = []
    nets = []

    for _, g in genomes:
      net = neat.nn.FeedForwardNetwork.create(g, config)
      nets.append(net)
      agents.append(neat_agent())
      g.fitness = 0
      ge.append(g)

    h = random.randint(1, 50)
    w = random.randint(1, 50)

    num_agents = len(agents)
    for i in range(num_agents):
      for j in range(i + 1, i + 3): # next 5 agents compare
        if (j >= num_agents):
           j -= num_agents
        agent_1 = agents[i]
        agent_2 = agents[j]
        test_game = cgol(agent_1, agent_2,board_h = h,board_w = w, max_rounds = 50, map = np.random.binomial(1,0.1,(h,w)))
        fitness_1, fitness_2 = test_game.eval_genome(nets[i], nets[j])
        ge[i].fitness = ge[i].fitness + fitness_1
        ge[j].fitness = ge[j].fitness + fitness_2
    
    for i in range(num_agents):
      for j in range(i + 1, i + 3): # next 5 agents compare
        if (j >= num_agents):
           j -= num_agents
        agent_1 = agents[i]
        agent_2 = rand_agent()
        test_game = cgol(agent_1, agent_2,board_h = h,board_w = w, max_rounds = 50, map = np.random.binomial(1,0.1,(h,w)))
        fitness_1, fitness_2 = test_game.eval_genome(nets[i], nets[j])
        ge[i].fitness = ge[i].fitness + fitness_1
    
    for i in range(num_agents):
      for j in range(i + 1, i + 3): # next 5 agents compare
        if (j >= num_agents):
           j -= num_agents
        agent_1 = agents[i]
        agent_2 = rand_enemy_agent()
        test_game = cgol(agent_1, agent_2,board_h = h,board_w = w, max_rounds = 50, map = np.random.binomial(1,0.1,(h,w)))
        fitness_1, fitness_2 = test_game.eval_genome(nets[i], nets[j])
        ge[i].fitness = ge[i].fitness + fitness_1

    idx = 0
    max = 0
    # Find the index of the maximum value
    for i in range(len(genomes)):
        g = genomes[i][1]
        if (g.fitness > max):
            max = g.fitness
            idx = i - 1
    
    
    print(idx)
    print(max)
    print(len(nets))
    print(len(genomes))
    
    file_path_string = "gen1.{}.pickle".format(GEN)
    pickle.dump(nets[idx],open(file_path_string, "wb"))
    if max > 50000 or GEN >= 50:
        pickle.dump(nets[idx],open("best1.pickle", "wb"))

In [38]:
if __name__ == "__main__":
  path = "C:\\Users\\Charles\\OneDrive\\Documents\\College\\Engineering_Competitions\\2024-3_MLDS_Long_Comp\\config_feedforward1.txt"
  file_path = os.path.normpath(path)
  print(file_path)
  run(file_path)

C:\Users\Charles\OneDrive\Documents\College\Engineering_Competitions\2024-3_MLDS_Long_Comp\config_feedforward1.txt

 ****** Running generation 0 ****** 

21
5794.4000000000015
40
40
Population's average fitness: -5681.31500 stdev: 4332.68122
Best fitness: 5794.40000 - size: (5, 12520) - species 1 - id 23
Average adjusted fitness: 0.402
Mean genetic distance 1.075, standard deviation 0.239
Population of 40 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    40   5794.4    0.402     0
Total extinctions: 0
Generation time: 198.049 sec

 ****** Running generation 1 ****** 

28
6792.700000000001
40
40
Population's average fitness: -15128.92000 stdev: 7235.57902
Best fitness: 6792.70000 - size: (6, 12460) - species 1 - id 68
Average adjusted fitness: 0.365
Mean genetic distance 1.092, standard deviation 0.231
Population of 40 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======

KeyboardInterrupt: 

In [ ]:
agent_1 = rand_agent()
agent_2 = rand_enemy_agent()


h = 20
w = 20
test_game = cgol(agent_1, agent_2,board_h = h,board_w = w, max_rounds = 50, map = np.random.binomial(1,0.1,(h,w)))

win_record, game_hist, action_hist = test_game.run_game(1, training=False)

# play the last episode/game
anim = anim_builder(game_hist[-1], action_hist[-1])

In [ ]:
#plot gif style
anim

In [ ]:
#save the gif for an individual game
anim.save('test_anim.gif', writer='pillow', fps=5)